In [5]:
import pandas as pd
import scanpy as sc
from math import acos, sqrt, pi, sin, cos
from scipy.io import mmread
from scipy import sparse
import numpy as np
from scipy.optimize import fsolve, root, basinhopping
import matplotlib.pyplot as plt
import h5py
import csv
import os
import loompy
import anndata
from pandas import HDFStore
from sklearn.manifold import TSNE
import collections
import scipy.sparse as sp_sparse
import tables
import plotly.express as px

In [2]:
genes = pd.read_csv("../Data/Genes.csv")
barcode_positions = pd.read_csv("../Data/SpotPositions.csv")
ScatterPies = pd.read_csv("../Data/SpotClusterMembership.csv")
GeneExpression = pd.read_csv("../Data/ClusterGeneExpression.csv")

In [3]:
merged_df = pd.merge(barcode_positions, ScatterPies, on="barcode")
merged_df

,barcode,x,y,radius,X1,X2,X3,X4,X5,X6,X7,X8,X9
0,AAACAAGTATCTCCCA-1,12709.0,10806.0,71.192911,0.176596,0.130452,0.000000,0.277719,0.000000,0.334386,0.000000,0.000000,0.080847
1,AAACACCAATAACTGC-1,3616.0,12513.0,71.192911,0.000000,0.000000,0.000000,0.000000,0.662506,0.000000,0.193410,0.144084,0.000000
2,AAACAGAGCGACTCCT-1,11839.0,3944.0,71.192911,0.000000,0.000000,0.330006,0.000000,0.000000,0.250267,0.000000,0.163155,0.256573
3,AAACAGGGTCTATATT-1,2961.0,10225.0,71.192911,0.000000,0.087536,0.000000,0.000000,0.463777,0.000000,0.237417,0.131194,0.080076
4,AAACATTTCCCGGATT-1,12159.0,12902.0,71.192911,0.197194,0.216818,0.000000,0.126936,0.099019,0.067587,0.292446,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3494,TTGTTGTGTGTCAAGA-1,9974.0,7183.0,71.192911,0.077686,0.000000,0.109485,0.372448,0.095749,0.344632,0.000000,0.000000,0.000000
3495,TTGTTTCACATCCAGG-1,6136.0,12325.0,71.192911,0.055007,0.083465,0.134945,0.000000,0.208435,0.121374,0.000000,0.124215,0.272560
3496,TTGTTTCATTAGTCTA-1,4821.0,12705.0,71.192911,0.000000,0.067899,0.000000,0.000000,0.545618,0.000000,0.161659,0.148889,0.075935
3497,TTGTTTCCATACAACT-1,4495.0,9846.0,71.192911,0.000000,0.000000,0.221199,0.000000,0.171150,0.191596,0.000000,0.175395,0.240660


In [6]:
columns = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9']
cumulativeAngledf = merged_df.copy()
def conditional_cumsum(row):
    cumsum = 0
    for col in columns:
        if row[col] != 0:
            cumsum += row[col]
            row[col] = cumsum
    return row
cumulativeAngledf[columns] = merged_df[columns].apply(conditional_cumsum, axis=1)
cumulativeAngledf

,barcode,x,y,radius,X1,X2,X3,X4,X5,X6,X7,X8,X9
0,AAACAAGTATCTCCCA-1,12709.0,10806.0,71.192911,0.176596,0.307048,0.000000,0.584767,0.000000,0.919153,0.000000,0.000000,1.0
1,AAACACCAATAACTGC-1,3616.0,12513.0,71.192911,0.000000,0.000000,0.000000,0.000000,0.662506,0.000000,0.855916,1.000000,0.0
2,AAACAGAGCGACTCCT-1,11839.0,3944.0,71.192911,0.000000,0.000000,0.330006,0.000000,0.000000,0.580273,0.000000,0.743427,1.0
3,AAACAGGGTCTATATT-1,2961.0,10225.0,71.192911,0.000000,0.087536,0.000000,0.000000,0.551313,0.000000,0.788730,0.919924,1.0
4,AAACATTTCCCGGATT-1,12159.0,12902.0,71.192911,0.197194,0.414012,0.000000,0.540948,0.639966,0.707554,1.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3494,TTGTTGTGTGTCAAGA-1,9974.0,7183.0,71.192911,0.077686,0.000000,0.187171,0.559619,0.655368,1.000000,0.000000,0.000000,0.0
3495,TTGTTTCACATCCAGG-1,6136.0,12325.0,71.192911,0.055007,0.138472,0.273416,0.000000,0.481851,0.603225,0.000000,0.727440,1.0
3496,TTGTTTCATTAGTCTA-1,4821.0,12705.0,71.192911,0.000000,0.067899,0.000000,0.000000,0.613517,0.000000,0.775176,0.924065,1.0
3497,TTGTTTCCATACAACT-1,4495.0,9846.0,71.192911,0.000000,0.000000,0.221199,0.000000,0.392350,0.583945,0.000000,0.759340,1.0


In [ ]:
hirescalef = 0.046594715
spotDiameter = 142.38582253945773
radius = spotDiameter * hirescalef / 2

In [17]:


# Function to calculate the area and solve for r1
def area(r1, instance, radius, ratio):
    Ang1 = np.arccos((r1*r1 + instance*instance - radius*radius) / (2 * r1 * instance))
    Ang2 = np.arccos((radius*radius + instance*instance - r1*r1) / (2 * radius * instance))
    area_value = r1*r1*Ang1 + radius*radius*Ang2 - r1*instance*np.sin(Ang1)
    return ratio * np.pi * radius**2 - area_value

# Iterate over each row in the DataFrame
for idx, row in cumulativeAngledf.iterrows():
    radius = row['radius']
    instance = np.sqrt(2) * radius
    print(f'index: {idx}')
    for i in range(1, 10):
        ratio = row[f'X{i}']
        if ratio != 0:
            # Use fsolve to find the root of the area function
            r1_solution = fsolve(area, radius, args=(instance, radius, ratio))[0]
            Ang1 = np.arccos((r1_solution*r1_solution + instance*instance - radius*radius) / (2 * r1_solution * instance))
            
            merged_df.at[idx, f'X{i}_radius'] = r1_solution
            merged_df.at[idx, f'X{i}_Ang1'] = Ang1
            print(f'X{i}_radius: {r1_solution}, X{i}_Ang1: {Ang1}')
        else:
            merged_df.at[idx, f'X{i}_radius'] = np.nan
            merged_df.at[idx, f'X{i}_Ang1'] = np.nan
            print(f'X{i}_radius: NaN, X{i}_Ang1: NaN')


index: 0
X1: 0.176595707865445
X1_radius: 70.46379021836357, X1_Ang1: 0.785345175335779
X2: 0.307048143372492
X2_radius: 87.3805484938377, X2_Ang1: 0.7641086301730473
X3: 0.0
X3_radius: NaN, X3_Ang1: NaN
X4: 0.584766805273809
X4_radius: 117.77953102104308, X4_Ang1: 0.6458021458537714
X5: 0.0
X5_radius: NaN, X5_Ang1: NaN
X6: 0.919153185918146
X6_radius: 71.19291126972887, X6_Ang1: 0.7853981633974483
X7: 0.0
X7_radius: NaN, X7_Ang1: NaN
X8: 0.0
X8_radius: NaN, X8_Ang1: NaN
X9: 0.9999999999999997
X9_radius: 71.19291126972887, X9_Ang1: 0.7853981633974483
index: 1
X1: 0.0
X1_radius: NaN, X1_Ang1: NaN
X2: 0.0
X2_radius: NaN, X2_Ang1: NaN
X3: 0.0
X3_radius: NaN, X3_Ang1: NaN
X4: 0.0
X4_radius: NaN, X4_Ang1: NaN
X5: 0.662506365258785
X5_radius: 125.92944084198491, X5_Ang1: 0.6001339329519291
X6: 0.0
X6_radius: NaN, X6_Ang1: NaN
X7: 0.8559159304167101
X7_radius: 147.3730349539776, X7_Ang1: 0.444865910093106
X8: 1.0
X8_radius: 71.19291126972887, X8_Ang1: 0.7853981633974483
X9: 0.0
X9_radius: NaN

/var/folders/pm/_db3z8953vb1gtk99jkk_sb80000gn/T/ipykernel_93801/1425021339.py:3: RuntimeWarning: invalid value encountered in arccos
  Ang1 = np.arccos((r1*r1 + instance*instance - radius*radius) / (2 * r1 * instance))
/var/folders/pm/_db3z8953vb1gtk99jkk_sb80000gn/T/ipykernel_93801/1425021339.py:4: RuntimeWarning: invalid value encountered in arccos
  Ang2 = np.arccos((radius*radius + instance*instance - r1*r1) / (2 * radius * instance))
/Users/siyuanzhao/anaconda3/envs/VDS/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:177: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


X2_radius: 66.15115960931789, X2_Ang1: 0.7826957833229061
X3: 0.0
X3_radius: NaN, X3_Ang1: NaN
X4: 0.0
X4_radius: NaN, X4_Ang1: NaN
X5: 0.570381940219688
X5_radius: 116.27327314639226, X5_Ang1: 0.653590967721317
X6: 0.6446756602059218
X6_radius: 124.05426611943446, X6_Ang1: 0.6111844279379025
X7: 0.0
X7_radius: NaN, X7_Ang1: NaN
X8: 0.7748612216644158
X8_radius: 138.01459893441887, X8_Ang1: 0.5200903014843146
X9: 0.9999999999999998
X9_radius: 71.19291126972887, X9_Ang1: 0.7853981633974483
index: 2871
X1: 0.327170865123522
X1_radius: 89.75709097495255, X1_Ang1: 0.7580549872541412
X2: 0.0
X2_radius: NaN, X2_Ang1: NaN
X3: 0.3959909327313941
X3_radius: 97.59422847180085, X3_Ang1: 0.7338887513840019
X4: 0.6273768297254071
X4_radius: 122.23986459958144, X4_Ang1: 0.6215618262007833
X5: 0.7125995660621189
X5_radius: 131.2434326769147, X5_Ang1: 0.5669238086629877
X6: 1.0000000000000009
X6_radius: 71.19291126972887, X6_Ang1: 0.7853981633974483
X7: 0.0
X7_radius: NaN, X7_Ang1: NaN
X8: 0.0
X8_radi